In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from library import datasets, models
from flax import linen as nn
from jax import random, numpy as jnp
import optax
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [16]:
MODEL_LATENT_DIM = 4
TRUE_LATENT_DIM = 1
AMBIENT_DIM = 2

In [17]:
taylor_pair_dis = datasets.point_dataset.PointPairDistribution.random_taylor(
    latent_dim=TRUE_LATENT_DIM,
    dis_A_dim=AMBIENT_DIM,
    dis_B_dim=AMBIENT_DIM,
    latent_range=1,
    max_order=2,
    coeff_range=1,
    noise_std_A=0.025,
    noise_std_B=0,
    key=random.PRNGKey(2))

dataset = taylor_pair_dis.generate_dataset(1000)

In [18]:
A, _ = dataset.get_all_point_pairs()

In [19]:
discriminator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(16),
    nn.relu,
    nn.Dense(16),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(1),
])

generator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(4),
    nn.relu,
    nn.Dense(AMBIENT_DIM),
])

model = models.vanilla_gan.VanillaGAN(
    generator=generator,
    discriminator=discriminator,
    latent_shape=(MODEL_LATENT_DIM,),
    ambient_shape=(AMBIENT_DIM,)
)

print(model)
model.initialize(loss_fn=optax.sigmoid_binary_cross_entropy)

generator:
	Model Structure:
		Sequential(
		    # attributes
		    layers = [Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7fbe57845480>, Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7fbe57845480>, Dense(
		        # attributes
		        features = 4
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7fbe57845480>, Dense(
		        # attributes
		        

In [26]:
model.train(datasets.base.TensorDataset(A), optax.adam(learning_rate=1e-2), print_every=5, batch_size=1000, num_epochs=250)

iteration 0; gen_loss:  6.90e-01; dis_loss:  7.05e-01; gen_grads_magnitude:  2.87e-03; dis_grads_magnitude:  7.08e-03: : 1it [00:05,  5.05s/it]
iteration 0; gen_loss:  6.84e-01; dis_loss:  6.98e-01; gen_grads_magnitude:  5.20e-04; dis_grads_magnitude:  3.48e-03: : 1it [00:00,  7.57it/s]
iteration 0; gen_loss:  6.81e-01; dis_loss:  6.96e-01; gen_grads_magnitude:  1.84e-05; dis_grads_magnitude:  2.65e-03: : 1it [00:00,  6.65it/s]
iteration 0; gen_loss:  6.80e-01; dis_loss:  6.94e-01; gen_grads_magnitude:  2.62e-07; dis_grads_magnitude:  1.79e-03: : 1it [00:00,  6.79it/s]
iteration 0; gen_loss:  6.80e-01; dis_loss:  6.93e-01; gen_grads_magnitude:  0.00e+00; dis_grads_magnitude:  1.17e-03: : 1it [00:00,  6.86it/s]
iteration 0; gen_loss:  6.81e-01; dis_loss:  6.93e-01; gen_grads_magnitude:  0.00e+00; dis_grads_magnitude:  9.35e-04: : 1it [00:00,  6.64it/s]
iteration 0; gen_loss:  6.82e-01; dis_loss:  6.93e-01; gen_grads_magnitude:  0.00e+00; dis_grads_magnitude:  8.94e-04: : 1it [00:00,  6.

In [27]:
gan_distribution = model.create_distribution()

In [28]:
true_samples = A

In [29]:
fake_samples = gan_distribution.draw_samples(random.uniform(random.PRNGKey(0), (1000, MODEL_LATENT_DIM), minval=-1, maxval=1))
# fake_samples = gan_distribution.draw_samples(random.normal(random.PRNGKey(0), (1000, LATENT_DIM)))

In [30]:
print(true_samples.shape, fake_samples.shape)

(1000, 2) (1000, 2)


In [31]:
df = {'x': jnp.concatenate((true_samples[:, 0], fake_samples[:, 0]), axis=0),
      'y': jnp.concatenate((true_samples[:, 1], fake_samples[:, 1]), axis=0),
      'labels': jnp.concatenate((jnp.ones((true_samples.shape[0],)), jnp.zeros((fake_samples.shape[0],))), axis=0)}

In [32]:
px.scatter(df, x='x', y='y', color='labels')